# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [12]:
import pandas as pd
import pymongo
from pymongo import MongoClient

In [13]:
client = pymongo.MongoClient()
client = MongoClient("mongo")
database = client['exercices']
collection = database['youtube']
print(client.database_names())

['admin', 'config', 'exercices', 'local', 'series']


In [14]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube

video_id                                              title  \
0     XpVt6Z1Gjjo  1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...   
1     K4wEI5zhHB0            iPhone X — Introducing iPhone X — Apple   
2     cLdxuaxaQwc                                        My Response   
3     WYYvHb03Eog                          Apple iPhone X first look   
4     sjlHnJvXdQs                                  iPhone X (parody)   
5     cMKX2tE5Luk    The Disaster Artist | Official Trailer HD | A24   
6     8wNr-NQImFg       The Check In: HUD, Ben Carson and Hurricanes   
7     _HTXMhKWqnA                   iPhone X Impressions & Hands On!   
8     _ANP3HR1jsM                         ATTACKED BY A POLICE DOG!!   
9     zgLtEob6X-Q                 Honest Trailers - The Mummy (2017)   
10    Ayb_2qbZHm4                                Honest College Tour   
11    CsdzflTXBVQ  Best Floyd Mayweather Interview | Awkward Puppets   
12    l864IBj7cgw  Jennifer Lawrence Challenges Jimmy to an Axe T...   
13    4MkC65emkG4  Hand In Hand A Benefit For Hurricane Relief | MTV   
14    vu_9muoxT50  Colin Cloud: Mind Reader Predicts Your Tweets ...   
15    1L7JFN7tQLs    iPhone X Hands on - Everything you need to know   
16    ZQK1F0wz6z4                          What Do You Want to Eat?!   
17    T_PuZBdT2iM  getting into a conversation in a language you ...   
18    w8fAellnPns  Juicy Chicken Breast - You Suck at Cooking (ep...   
19    UCrBICYM0yM  Downsizing (2017) - Official Trailer - Paramou...   
20    -Ifnaxi2LQg          Fergie - You Already Know ft. Nicki Minaj   
21    B7YaMkCl3XA      Hurricane Irma death toll rises to 10 in Cuba   
22    5ywKal6-anc  Gigi Hadid Loses High Heel During Fashion Week...   
23    4Yue-q9Jdbk     SUPERFRUIT REACTS TO TEENS REACT TO SUPERFRUIT   
24    JhA1Wi9mrns         Kid Rock - Tennessee Mountain Top [Lyrics]   
25    EVp4-qjWVJE  Chargers vs. Broncos | NFL Week 1 Game Highlights   
26    LcZ2AuvxXNA                Nerf Bow Trick Shots | Dude Perfect   
27    MdzGZv3zQ-U  Action Bronson and Sean Evans Have a Sandwich ...   
28    1ZNZY-gd3K0  Joe Arpaio: Last Week Tonight with John Oliver...   
29    2wxyDrfwlXQ               What's In This 90s Mystery Caboodle?   
...           ...                                                ...   
7962  k9VmfRBkUuM                          Crane Removal - Timelapse   
7963  3y5A4paFOb4  Logan Paul - Outta My Hair [Official Music Video]   
7964  x4SheWJkn6s          Latinos React To The Selena Google Doodle   
7965  IOp8n5TzcSQ           Bailey Bryan - Too Good [Official Cover]   
7966  8Wb9KYnNxUc  Draymond Green: I don't know how serious the R...   
7967  adgAxnflvOE  EXCITED Dogs in Car Can't Stop Howling For The...   
7968  mPjgRKW_Jmk  Equifax: Last Week Tonight with John Oliver (HBO)   
7969  XPqko6Q0lpE                     John Oliver - Harvey Weinstein   
7970  AApylcoF9jA            Storm Ophelia makes landfall in Ireland   
7971  xTyTQgb-y-A  Jordan​ and​ ​Lindsay’s - Foxtrot - Dancing wi...   
7972  CEdAjI801Wo  How Jonathan Groff Got The Lead On Netflix's  ...   
7973  PeUgWK0pUIA            How Video Games are Turning into Movies   
7974  s8GrGvjhz0s  Blake Lively - Talks About Harvey Weinstein - GMA   
7975  RA9gg4qBb30                 John McCain Liberty Medal speech -   
7976  X2tejP45Lwc  28 Things I'd Tell My Younger Self! Life Lesso...   
7977  z2v_rL5C8C8  VINTAGE SHOPPING FOR THE FIRST TIME! (a mess lol)   
7978  -_Q5kO4YXFs  My baby hears me for the first time and is alm...   
7979  xGd8KIXqPJY  Katy Perry - Making Of “Swish Swish” Music Vid...   
7980  n530GFrdPY8                        Brad Paisley - Heaven South   
7981  3VSa-oARk-w  Monument Valley 2 - Available on Android & iOS...   
7982  Hlt3rA-oDao                                    Kellywise - SNL   
7983  Ni5Uc-DdMqk  Chris Hemsworth and Mark Ruffalo Australian In...   
7984  pfAKloMu4S0  What the Bannon vs. McConnell fight means for ...   
7985  CN4YL1KPDAA                           Pixel

## Question 0

### Netoyer les données

In [15]:
df_youtube = pd.read_csv("./data/USvideos.csv")

In [16]:
df_youtube['date'] = pd.to_datetime(df_youtube['date'], errors='coerce')
df_youtube = df_youtube.dropna(subset=['date'])

 


df_youtube.rename(columns={'video_id':'_id'}, inplace=True)

 

df_youtube = df_youtube.drop_duplicates(subset='_id', keep='last')

 

l_d =df_youtube.to_dict(orient='records')

 

import json
with open('./data/US_category_id.json', 'r') as f:
    categ = json.load(f)

 

for video in l_d:
    for cat in categ['items']:
        if video['category_id'] == int(cat['id']):
            video['category_name'] = cat['snippet']['title']

### Importer les données

In [17]:
df_youtube

_id                                              title  \
22    5ywKal6-anc  Gigi Hadid Loses High Heel During Fashion Week...   
33    GGm0FQ6i74U                 What Hillary Clinton really thinks   
42    qJJHhVf3_ZM  Gwyneth Paltrow Crashes James' Goop Magazine Rant   
49    9CILSvf1snQ                 GOLD DIVA MACARONS- The Scran Line   
71    k9bYO7uSnrg                                  iPhone X hands on   
124   BRzPaDMeRks     These Are The Best Features Coming To iPhones.   
127   -tCu0tKrzhc  Sloane Stephens jokes that $3.7 million check ...   
153   WCKJPn1UcXc           One America Appeal: Our Friends In Texas   
154   TDskW3JvTAY  ANVIL Vs. WATERMELON PYRAMID! (CRAZY 45m DROP ...   
156   U2CqZNd6rgM                         Welcome to JURASSIC FALLS!   
161   FTIStPerVNw  Bernie Sanders Tries To Say Something Nice Abo...   
162   AX8-YzMKZhQ  Sam Smith - Too Good At Goodbyes (Official Audio)   
164   c3pFWJCQy5E                   Update: Problem with Chemo Pills   
165   eJB1gcydsbU               If kids shows were honest about life   
166   AR4UgRJOUQY       What Does Your Search History Say About You?   
167   DGlYMJRwevI  Nerf meets Call of Duty: Gun Game 8.0 | First ...   
168   PGfSaVDymjk  G-Eazy - No Limit (Audio) ft. A$AP Rocky, Cardi B   
169   a2B9pA8aGUc                  Rutgers Falls To Eastern Michigan   
170   4Ltc4HO3NIE  Florida Weather Cameras - Hurricane Irma (19 c...   
171   A-fofQ9VpPQ      Alias Grace | Official Trailer [HD] | Netflix   
172   pCSgWJzEvrM            The only dating advice you'll ever need   
173   bnON1y-fKag                          Home Again - Movie Review   
174   VCVjbrt30tY                            Fifth Harmony - Deliver   
175   2caEL5ATPX4  Janet Jackson performs Dammn Baby & I Get Lone...   
176   Ie50Nw9NkOk  Jordan Peele and Spike Lee Team Up for Movie A...   
177   tt2k8PGm-TI                      ZAYN - Dusk Till Dawn ft. Sia   
178   mGqR9sgMIyA                  Orlando braces for Hurricane Irma   
179   D1KpuWg2hc0  IT's Nicholas Hamilton & Chosen Jacobs Play RA...   
180   qh7QJ_jLam0  Green Day - Too Dumb to Die (Official Lyric Vi...   
181   XovZ04gmQgE  Hailee Steinfeld & Alesso (ft. Florida Georgia...   
...           ...                                                ...   
7962  k9VmfRBkUuM                          Crane Removal - Timelapse   
7963  3y5A4paFOb4  Logan Paul - Outta My Hair [Official Music Video]   
7964  x4SheWJkn6s          Latinos React To The Selena Google Doodle   
7965  IOp8n5TzcSQ           Bailey Bryan - Too Good [Official Cover]   
7966  8Wb9KYnNxUc  Draymond Green: I don't know how serious the R...   
7967  adgAxnflvOE  EXCITED Dogs in Car Can't Stop Howling For The...   
7968  mPjgRKW_Jmk  Equifax: Last Week Tonight with John Oliver (HBO)   
7969  XPqko6Q0lpE                     John Oliver - Harvey Weinstein   
7970  AApylcoF9jA            Storm Ophelia makes landfall in Ireland   
7971  xTyTQgb-y-A  Jordan​ and​ ​Lindsay’s - Foxtrot - Dancing wi...   
7972  CEdAjI801Wo  How Jonathan Groff Got The Lead On Netflix's  ...   
7973  PeUgWK0pUIA            How Video Games are Turning into Movies   
7974  s8GrGvjhz0s  Blake Lively - Talks About Harvey Weinstein - GMA   
7975  RA9gg4qBb30                 John McCain Liberty Medal speech -   
7976  X2tejP45Lwc  28 Things I'd Tell My Younger Self! Life Lesso...   
7977  z2v_rL5C8C8  VINTAGE SHOPPING FOR THE FIRST TIME! (a mess lol)   
7978  -_Q5kO4YXFs  My baby hears me for the first time and is alm...   
7979  xGd8KIXqPJY  Katy Perry - Making Of “Swish Swish” Music Vid...   
7980  n530GFrdPY8                        Brad Paisley - Heaven South   
7981  3VSa-oARk-w  Monument Valley 2 - Available on Android & iOS...   
7982  Hlt3rA-oDao                                    Kellywise - SNL   
7983  Ni5Uc-DdMqk  Chris Hemsworth and Mark Ruffalo Australian In...   
7984  pfAKloMu4S0  What the Bannon vs. McConnell fight means for ...   
7985  CN4YL1KPDAA                           Pixel 2 Cl

In [18]:
from pymongo import MongoClient

client = MongoClient("mongo:27017")

db_yt = client.yt

collection_yt = db_yt['videos']

collection_yt.delete_many({})

collection_yt.insert_many(l_d)

In [20]:
l_d

[{'_id': '5ywKal6-anc',
  'title': 'Gigi Hadid Loses High Heel During Fashion Week, Walks It Off Like a Pro | TMZ',
  'channel_title': 'TMZ',
  'category_id': 24,
  'tags': 'TMZ2016FS11221|TMZ|Hollywood|Celebrity|Entertainment|Famous|Hollywood News|Fame|Entertainment News',
  'views': 703750,
  'likes': 2921,
  'dislikes': 2196,
  'comment_total': 1042,
  'thumbnail_link': 'https://i.ytimg.com/vi/5ywKal6-anc/default.jpg',
  'date': Timestamp('1970-01-01 00:00:00.000000013'),
  'category_name': 'Entertainment'},
 {'_id': 'GGm0FQ6i74U',
  'title': 'What Hillary Clinton really thinks',
  'channel_title': 'Vox',
  'category_id': 25,
  'tags': 'vox.com|vox|explain|ezra klein|hillary clinton|what happened|donald trump|election 2016|election 2018|bernie sanders|vox conversations|hillary clinton book|hillary clinton interview|hillary clinton book interview|what happened book|clinton|hillary|maga|make america great again|universal basic income|universal health care|single-payer|health care|hill

## Question 1  

In [21]:
cur = collection_yt.find({"channel_title":"Apple"})
list(cur)

[{'_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
  'views': 19707391,
  'likes': 381919,
  'dislikes': 69465,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': datetime.datetime(1970, 1, 1, 0, 0),
  'category_name': 'Science & Technology'},
 {'_id': 'A9k88sMyiJM',
  'title': 'Apple Watch Series 3 + Apple Music — Roll — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|airpods|apple watch|apple watch series 3|apple watch cellular|apple watch 4g|apple music|40 million songs on your wrist|apple skateboard ad|apple train station ad|apple roll ad|apple watch skateboard ad|apple watch train station ad|apple watch roll ad|airpods skateboard ad|airpods train station ad|airpods roll ad|apple music skat

## Question 2

In [25]:
cur = collection_yt.aggregate([
    { "$group": { 
        "_id": "$category_id"
        }  
    },
    { "$group": 
         { 
             "_id": "category_id",
              "count": { "$sum": 1 } 
         } 
    } 
])
list(cur)

[{'_id': 'category_id', 'count': 16}]

## Question 3

## Question 4

In [27]:
cur = collection_yt.find().sort([("views",-1)]).limit(3)
list(cur)

[{'_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': datetime.datetime(1970, 1, 1, 0, 0),
  'category_name': 'Music'},
 {'_id': 'tt2k8PGm-TI',
  'title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'channel_title': 'ZaynVEVO',
  'category_id': 10,
  'tags': 'Dusk Till Dawn|Pop|RCA Records Label|ZAYN feat. Sia',
  'views': 36323498,
  'likes': 1431683,
  'dislikes': 28049,
  'comment_total': 100661,
  'thumbnail_link': 'https://i.ytimg.com/vi/tt2k8PGm-TI/default.jpg',
  'date': datetime.datetime(1970, 1, 1, 0, 0),
  'category_name': 'Music'},
 {'_id': 'LunHybOKIjU',
  'title': 'Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher',
  'channel_title': 'BETNetworks',
  'category_id': 24,
  'tags': 'BET|BET Netwo

## Question 5

In [58]:
result = collection_yt.aggregate([{
    "$group": {
        "_id": "$category_id",
        "moyenne_vues_par_cat": {"$avg": "$views"}
    }
}])

for line in result:
    print(line)

{'_id': 25, 'moyenne_vues_par_cat': 533575.1262626263}
{'_id': 10, 'moyenne_vues_par_cat': 1442240.943283582}
{'_id': 29, 'moyenne_vues_par_cat': 681178.2}
{'_id': 17, 'moyenne_vues_par_cat': 618074.711409396}
{'_id': 15, 'moyenne_vues_par_cat': 633906.4210526316}
{'_id': 26, 'moyenne_vues_par_cat': 556002.6679389313}
{'_id': 28, 'moyenne_vues_par_cat': 861563.6158940397}
{'_id': 23, 'moyenne_vues_par_cat': 1317200.7289719626}
{'_id': 24, 'moyenne_vues_par_cat': 1059427.7735849055}
{'_id': 20, 'moyenne_vues_par_cat': 777213.0357142857}
{'_id': 19, 'moyenne_vues_par_cat': 426492.22222222225}
{'_id': 22, 'moyenne_vues_par_cat': 892992.28515625}
{'_id': 1, 'moyenne_vues_par_cat': 966547.2871287129}
{'_id': 2, 'moyenne_vues_par_cat': 622727.8648648649}
{'_id': 43, 'moyenne_vues_par_cat': 8726.0}
{'_id': 27, 'moyenne_vues_par_cat': 550585.2659574468}


## Question 6 

In [99]:
result = collection_yt.aggregate([{
    "$group": {
        "_id": "$channel_title",
        "moyenne_likes": {"$avg": "$likes"}
    }
},

    { "$sort": { "moyenne_likes": -1 } },
    { "$limit": 10 }])

list(result)

[{'_id': 'ibighit', 'moyenne_likes': 1533457.0},
 {'_id': 'ZaynVEVO', 'moyenne_likes': 1431683.0},
 {'_id': 'BETNetworks', 'moyenne_likes': 982842.0},
 {'_id': 'melanie martinez', 'moyenne_likes': 921577.0},
 {'_id': 'jypentertainment', 'moyenne_likes': 793387.0},
 {'_id': 'Logan Paul Vlogs', 'moyenne_likes': 724478.5},
 {'_id': 'Wengie', 'moyenne_likes': 656526.0},
 {'_id': 'Alesso', 'moyenne_likes': 618436.0},
 {'_id': 'shakiraVEVO', 'moyenne_likes': 578266.0},
 {'_id': 'TaylorSwiftVEVO', 'moyenne_likes': 508015.0}]